In [1]:
# import some keras and basic module 

from __future__ import print_function 
import tensorflow.keras as keras

# data set cifar 10 
from tensorflow.keras.datasets import cifar10 

# basic preprocesssing for image data 
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# construct neural network 
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten 

from tensorflow.keras.layers import Conv2D, MaxPooling2D

import os 

import numpy as np 

import matplotlib.pyplot as plt

from tensorflow.keras.utils import plot_model
import tensorflow as tf 

# config = tf.ConfigProto()
# tf.enable_eager_execution(config=config)

AttributeError: module 'tensorflow' has no attribute 'ConfigProto'

In [ ]:

## basic parameters 
batch_size = 32 
num_classes = 10 
num_epochs = 300 
is_data_augmentation = True 
model_dir = 'models'
model_filename = 'BeatLeNet5_2_FMP.h'



In [ ]:
#read data and split data into train and test set

(train_set,train_label),(test_set,test_label) = cifar10.load_data() 

print('the shape of training data set is: ',train_set.shape) 

# print number of train and test samples 

# train samples 50000
#print(train_set.shape[0] , 'train samples') 

# test samples 10000
#print(test_set.shape[0] , 'test samples') 

#print(train_label)

In [ ]:
# Convert class label to binary vector 
train_label = keras.utils.to_categorical(train_label,num_classes)
test_label  = keras.utils.to_categorical(test_label,num_classes) 
print(train_label.shape)
print(test_label.shape)

In [ ]:
# build Max Fractional Pooling Network
def initialize_mfp_layer(X):
    mean = 0
    sigma = 2 
    conv_w = tf.Variable(tf.truncated_normal(shape = [2,2,1,160],mean = mean, stddev = sigma))
    conv_b = tf.Variable(tf.zeros(160))
    conv = tf.nn.conv2d(X,conv_w,strides = [1,1,1,1] ,padding = 'same')
    conv = tf.nn.relu(conv)
    pool = tf.nn.fractional_max_pool(conv,ksize = [1,2,2,1], strides = [1,2,2,1],padding ='same')
    return pool
def generate_mfp_next_layer(previous_layer, n):
    mean = 0
    sigma = 2 
    conv_w = tf.Variable(tf.truncated_normal(shape = [2,2,1,n*160],mean = mean, stddev = sigma))
    conv_b = tf.Variable(tf.zeros(n*160))
    conv = tf.nn.conv2d(previous_layer,conv_w,strides = [1,1,1,1] ,padding = 'same')
    conv = tf.nn.relu(conv)
    pool = tf.nn.fractional_max_pool(conv,ksize = [1,2,2,1], strides = [1,2,2,1],padding ='same')
    return pool
def mfp_Net(X,n):
      
    intialized_layer = initialize_mfp_layer(X)
    previous_layer = initialized_layer 
    for n = 2:12:
        previous_layer =generate_mfp_next_layer(previous_layer,n)
        
    
    return previous_layer

In [ ]:
x = tf.placeholder(tf.float32, (None, 32, 32, 1))
y = tf.placeholder(tf.int32, (None))
one_hot_y = tf.one_hot(y, 10)

In [ ]:

rate = 0.001

logits = mfp_Net(x)
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits = logits, labels = one_hot_y)
loss_operation = tf.reduce_mean(cross_entropy)
optimizer = tf.train.AdamOptimizer(learning_rate = rate)
training_operation = optimizer.minimize(loss_operation)

In [ ]:

correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(one_hot_y, 1))
accuracy_operation = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
saver = tf.train.Saver()

the code input for Conv2D 
```
tf.keras.layers.Conv2D(
    filters, kernel_size, strides=(1, 1), padding='valid', data_format=None,
    dilation_rate=(1, 1), activation=None, use_bias=True,
    kernel_initializer='glorot_uniform', bias_initializer='zeros',
    kernel_regularizer=None, bias_regularizer=None, activity_regularizer=None,
    kernel_constraint=None, bias_constraint=None, **kwargs
)
```

In [ ]:


# initialization of optimizer 
opt = keras.optimizers.SGD(learning_rate = 0.0001)

# train the model by optimizer
cnn_model.compile(loss = 'categorical_crossentropy',
              optimizer = opt,
              metrics = ['accuracy'])

train_set = train_set.astype('float32')
test_set = test_set.astype('float32') 

train_set /= 255 
test_set /= 255 

if not is_data_augmentation:
    print('Not using data augmentation.')
    cnn_history = cnn_model.fit(train_set, train_label,
              batch_size=batch_size,
              epochs=num_epochs,
              validation_data=(test_set, test_label),
              shuffle=True)
else:
    print('Using real-time data augmentation.')
    # This will do preprocessing and realtime data augmentation:
     
    
    # set parameter for data augmentation 
    data_transform_parameters = {
                                "featurewise_center":True,
                                "featurewise_std_normalization" : True,
                                "rotation_range": 20,
                                "width_shift_range":0.2,
                                "height_shift_range":0.2,
                                "horizontal_flip":True
                                }
    
    # get object of augmentation data generator 
    augment_data_set_generator = ImageDataGenerator(data_transform_parameters)    
    
    
    # get augmented data set 
    
    augment_train_set = augment_data_set_generator.apply_transform(train_set,data_transform_parameters)
    print(augment_train_set.shape)
  
    
    # Limit GPU device to the first GPU 
    gpus = tf.config.experimental.list_physical_devices('GPU')
    if gpus:
      # Restrict TensorFlow to only use the first GPU
      try:
        tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
      except RuntimeError as e:
        # Visible devices must be set before GPUs have been initialized
        print(e)
        
        
    # train model by GPU 
    with tf.device('/device:GPU:0'):
        cnn_history = cnn_model.fit(augument_train_set, train_label,
                  batch_size=batch_size,
                  epochs=num_epochs,
                  validation_data=(test_set, test_label),
                  shuffle=True)
    
# save model and weights 
if not os.path.isdir(model_dir): 
    os.makedirs(model_dir) 
model_path = os.path.join(model_dir,model_filename)
cnn_model.save(model_path)
print("CNN Model saved at %s " % model_path)

# Score trained model 

test_loss_value, test_metric_value = cnn_model.evaluate(test_set,test_label,verbose =1 )
train_loss_value, train_metric_value = cnn_model.evaluate(train_set,train_label,verbose =1)


print("Train loss: ", train_loss_value) 
print("Trian metric: ", train_metric_value)

print("Test loss: ", test_loss_value)
print("Test accuracy:", test_metric_value)
# print(cnn_history.history)


In [ ]:

# get history data
epoth_x = np.linspace(0, 10, num_epochs, endpoint=True)
train_loss = cnn_history.history['loss']
train_accuracy = cnn_history.history['accuracy']
test_loss = cnn_history.history['val_loss']
test_accuracy = cnn_history.history['val_accuracy']

# plot train loss and accuracy
plt.figure() 
plt.plot(epoth_x, train_loss,label = 'train loss')
plt.plot(epoth_x,train_accuracy,label = 'train accuracy')
plt.title("train loss and train accuracy")
plt.legend()


# plot test loss and accuracy
plt.figure() 
plt.plot(epoth_x, test_loss, label = 'test loss')
plt.plot(epoth_x,test_accuracy, label = 'test accuracy')
plt.legend()
plt.title("test loss and test accuracy")